In [1]:
import numpy as np
import pandas as pd
from sunpy.physics.differential_rotation import rot_hpc
from astropy import units as u
from shapely.geometry import Polygon, Point
from shapely import wkt
import os
import datetime
from sunpy.time import parse_time

In [6]:
def collect_ar_noaa(df, folder):
    unique_noaa = df.ar_noaanum.unique()
    keywords_ar = list(np.genfromtxt('keywords_ar.csv', delimiter=',', dtype =str))
    noaa_dic = {elem : pd.DataFrame for elem in unique_noaa}
    start_times = []
    end_times= []
    file_names = []
    i = 0
    for key in noaa_dic.keys():
        if i != 0:
            noaa_dic[key] = df[:][df.ar_noaanum == key]
            start_time = noaa_dic[key]['event_starttime'].values[0]
            end_time = noaa_dic[key]['event_endtime'].values[-1]
            start_times.append(start_time)
            end_times.append(end_time)
            fName = str(key)[:-2] +'_'+ start_time[0:10]+'_'+end_time[0:10]
            location = folder + '/' + fName+ '.csv'
            noaa_dic[key].to_csv(location, columns = keywords_ar, index = False)
            file_names.append(fName)
        i+=1  

In [7]:
ar_keywords = np.genfromtxt('keywords_ar_1.csv', delimiter = ',', dtype = str)

In [8]:
def merge_ar_sources(fname, ar_keywords, output_directory):
    inputFile = 'AR_raw/'+fname
    ar_set =pd.read_csv(inputFile, delimiter = ',', header = 0)
    ar_set_NOAA = ar_set.ix[ar_set['frm_identifier']=='NOAA SWPC']
    listOindices = ar_set_NOAA.index.tolist()
    keywords_ar_add = ['ar_mcintoshcls', 'ar_mtwilsoncls', 'ar_numspots']
    ar_set.sort_values(by=['event_starttime','frm_identifier'], ascending =[True, False])
    if listOindices!= [] and listOindices!=range(0, ar_set.shape[0]):
        for i in range(0, listOindices[0]):
            for elem in keywords_ar_add:
                ar_set.loc[i, elem] = ar_set.loc[listOindices[0], elem] 
        for i in range(listOindices[-1], ar_set.shape[0]):
            for elem in keywords_ar_add:
                ar_set.loc[i, elem] = ar_set.loc[listOindices[-1], elem]
        for i, idx in enumerate(listOindices):
            if idx!=listOindices[-1]:
                for j in range(idx, listOindices[i+1]):
                    for elem in keywords_ar_add:
                        ar_set.loc[j, elem] = ar_set.loc[idx, elem]
        ar_set = ar_set.drop(listOindices)
        outputFile = output_directory + '/' + fname
        ar_set.to_csv(outputFile, index = False, columns = ar_keywords)
        
    return ar_set

In [9]:
AR_folder = 'AR4'
for root, dirs, files in os.walk('AR_raw'):
    for f in files:
        if f[0] != '.':
            _ = merge_ar_sources(f, ar_keywords, AR_folder)

In [2]:
def rotate_polygon(polygon, start_time, end_time):
    poly_coords = list(polygon.exterior.coords)
    poly_coords = (np.asarray(poly_coords[0:-1])) *u.arcsec
    poly_coords_x = poly_coords[:,0]
    poly_coords_y = poly_coords[:,1]
    rotated_poly_coords_x, rotated_poly_coords_y = rot_hpc(poly_coords_x, poly_coords_y, start_time, end_time)
    return poly_coords_x.value, poly_coords_y.value, rotated_poly_coords_x.value, rotated_poly_coords_y.value

def get_poly_coords(x, y, left_side = False):
    if left_side: diff = abs(y-y[x.argmin()])
    else: diff = abs(y-y[x.argmax()])
    d = {'x': x, 'y': y, 'diff':diff}
    sorter_df = pd.DataFrame(data=d)
    sorter_df = sorter_df.sort_values(by='x', ascending = left_side)
    x0 = sorter_df['x'].values[0]
    y0 = sorter_df['y'].values[0]
    if sorter_df['diff'].values[1] >= sorter_df['diff'].values[2]:
        x1 = sorter_df['x'].values[1]
        y1 = sorter_df['y'].values[1]
    else:
        x1 = sorter_df['x'].values[2]
        y1 = sorter_df['y'].values[2]
    return x0, y0, x1, y1

#function breaks down when AR rotating around the limb

def rotate_ar(polygon, start_time, end_time):
    og_x, og_y, rot_x, rot_y = rotate_polygon(polygon, start_time, end_time)
    x0, y0, x1, y1 = get_poly_coords(og_x, og_y, True)
    x2, y2, x3, y3 = get_poly_coords(rot_x, rot_y, False)
    #ensure that vertices of quadrilateral are in counterwise order in order to create proper Polygon object
    if y0 < y1:
        dummy_y = y0
        dummy_x = x0
        y0 = y1
        x0 = x1
        y1 = dummy_y
        x1 = dummy_x
    if y2 > y3:
        dummy_y = y2
        dummy_x = x2
        y2 = y3
        x2 = x3
        y3 = dummy_y
        x3 = dummy_x
    rotated_ar_polygon = Polygon([(x0,y0), (x1,y1), (x2,y2), (x3,y3)])
    return rotated_ar_polygon

In [ ]:
for root, dirs, files in os.walk(AR_folder):
    for f in files:
        if f[0] != '.':
            inputFile = AR_folder +'/'+f
            ars = pd.read_csv(inputFile, delimiter = ',', header = 0)
            ar_polygon = ars['hpc_bbox']
            start_time = ars['event_starttime']
            end_time = ars['event_endtime']
            full_hpc_polygon = []
            for i in range(ars.shape[0]):
                poly = rotate_ar(ar_polygon[i], start_time[i], end_time[i])
                full_hpc_polygon.append(poly)
            

In [ ]:
def create_ar_list(AR_folder, outputF):
    file_names = []
    ar_nums = []
    start_years = []
    start_months = []
    start_days =[]
    start_times = []
    end_years = []
    end_months = []
    end_days = []
    end_times = []

    for root, dirs, files in os.walk(AR_folder):
        for f in files:
            if f[0] != '.':
                file_names.append(f)
                ar_num = f[0:5]
                ar_nums.append(ar_num)
                start_year = f[6:10]
                start_years.append(start_year)
                start_month = f[11:13]
                start_months.append(start_month)
                start_day = f[14:16]
                start_days.append(start_day)
                start_time = f[6:16]
                start_times.append(start_time)
                end_year = f[17:21]
                end_years.append(end_year)
                end_month = f[22:24]
                end_months.append(end_month)
                end_day = f[25:27]
                end_days.append(end_day)
                end_time = f[17:27]
                end_times.append(end_time)

    dic4df = {'file_name': file_names, 'ar_num': ar_nums,'start_year': start_years,'start_month': start_months,
              'start_day': start_days,'start_time': start_times, 'end_year': end_years,'end_month':end_months,
              'end_day': end_days,'end_time': end_times}
    ar_list = pd.DataFrame(data=dic4df)
    ar_list.to_csv(outputF, index = False)
        

In [ ]:
f = '11082_2010-06-14_2010-06-26.csv'
ar_num = f[0:5]
start_year = f[6:10]
start_month = f[11:13]
start_day = f[14:16]
start_time = f[6:16]
end_year = f[17:21]
end_month = f[22:24]
end_day = f[25:27]
end_time = f[17:27]
print ar_num, start_year, start_month, start_day, end_year, end_month, end_day, start_time, end_time

In [ ]:
create_ar_list('AR_test', 'ar_test_list.csv')

In [ ]:
'flare_search_2014-01-06_2014-01-07_60_100.csv'

In [15]:
def associate_ar(inputFile_fl, inputFile_ar, ar_folder, output2file=False, out_file = None):
    #import a record of flare events as a DataFrame
    flare_set = pd.read_csv(inputFile_fl, delimiter = ',', header = 0)
    #import a list of ar events as a DataFrame
    ar_list = pd.read_csv(inputFile_ar, delimiter = ',', header = 0)
    
    
    #how many flare events working with
    length = flare_set.shape[0]
    #list of zeroes with length of number of flare events
    zeroes = [0 for i in range(length)]
    #list of nulls with length of number of flare events
    nones = [None for i in range(length)]
    #create columns (filled with zeroes) for tracking associated events
    flare_set.loc[:, 'associated_ar'] = nones
    #get a list of ar keywords of relevance to flare events
    ar_keywords = list(np.genfromtxt('keywords_ar_append_fl.csv', delimiter=',', dtype=str))
    #create columns filled with Nones for each ar keyword
    for elem in ar_keywords:
         flare_set.loc[:, elem] = nones
    
    #convert start and end times to datetime objects
    flare_set['event_starttime'] = map(parse_time, flare_set['event_starttime'])
    flare_set['event_endtime'] = map(parse_time, flare_set['event_endtime'])
    ar_list['start_time'] = map(parse_time, ar_list['start_time'])
    ar_list['end_time'] = map(parse_time, ar_list['end_time'])
    
    #set positional row index 
    i = -1

    for annoying_wrong_obj_type in flare_set['event_starttime']:
        i += 1
        #print which flare event function is currently processing, so the user has an idea of how much longer
        #program will need to run
        start = flare_set['event_starttime'].values[i]
        end = flare_set['event_endtime'].values[i]
        print '%d / %d events' %((i+1), length)
        #begin eliminating ar events based on temporal parameters
        ar_search = ar_list.ix[ar_list['start_time']<=start]
        ar_search = ar_search.ix[ar_search['end_time']>=end]
        ar_files = list(ar_search['file_name'])
        
        #as long as the temporal search does not eliminate all possible related AR events, proceed
        if len(ar_files)!= 0:
            for f in ar_files:
                fname = ar_folder+'/'+f
                specific_ar = pd.read_csv(fname, delimiter = ',', header = 0)
                specific_ar = specific_ar.rename(columns={'hpc_bbox': 'ar_hpc_bbox'})
                found_time_match = False
                specific_ar['event_starttime'] = map(parse_time, specific_ar['event_starttime'])
                specific_ar['event_endtime'] = map(parse_time, specific_ar['event_endtime'])
                j = 0
                while found_time_match==False and j<len(specific_ar):
#                     print type(specific_ar['event_starttime'].values[j])
#                     print type(start)
#                     print type(specific_ar['event_endtime'].values[j])
                    if (specific_ar['event_starttime'].values[j] <= start and 
                        specific_ar['event_endtime'].values[j] >= start):
                        found_time_match == True
#                         print start
#                         print end
#                         print annoying_wrong_obj_type
                        fl_point = Point((flare_set['hpc_x'].values[i], flare_set['hpc_y'].values[i]))
                        ar_poly_og = wkt.loads(specific_ar['ar_hpc_bbox'].values[j])
                        end = specific_ar.loc[j, 'event_endtime']
                        ar_poly = rotate_ar(ar_poly_og, annoying_wrong_obj_type, end)
                        if fl_point.intersects(ar_poly):
                            flare_set['associated_ar'].values[i] = str(specific_ar.loc[j, 'SOL_standard'])
                            for elem in ar_keywords:
                                flare_set.loc[i, elem] = specific_ar[elem].values[j]
                    j+=1
            
    #create boolean var to easily determine whether flare associated with an AR
    k = 0
    is_ar = [0 for i in range(flare_set.shape[0])]
    for elem in flare_set['associated_ar']:
        if elem!=None:
            is_ar[k] = 1
        k+=1
    flare_set.loc[:, 'is_ar'] = is_ar
    
    #write dataframe to a csv file depending on initial parameters
    if output2file:
        if out_file == None:
            #create a generic name for file based on search parameters if no file name specified
             out_file = inputFile_fl[0:-4]+'_with_ar.csv'
        #import which keywords to keep for outported data
        flare_keywords = list(np.genfromtxt('keywords_flare_after_merge.csv', delimiter=',', dtype=str))
        #add to these keywords descriptors of associated AR
        flare_keywords.extend(['is_ar','associated_ar'])
        flare_keywords.extend(ar_keywords)
        #write to csv
        flare_set.to_csv(path_or_buf=out_file, columns = flare_keywords, index = False)
        
    return flare_set

In [14]:
associate_ar('flare_tester_cleaned.csv', 'ar_test_list.csv', 'AR_test', output2file=True, out_file = None)

1 / 32 events
2 / 32 events
3 / 32 events
4 / 32 events
5 / 32 events
6 / 32 events
7 / 32 events
8 / 32 events
9 / 32 events
10 / 32 events
11 / 32 events
12 / 32 events
13 / 32 events
14 / 32 events
15 / 32 events
16 / 32 events
17 / 32 events
18 / 32 events
19 / 32 events
20 / 32 events
21 / 32 events
22 / 32 events
23 / 32 events
24 / 32 events
25 / 32 events
26 / 32 events
27 / 32 events
28 / 32 events
29 / 32 events
30 / 32 events
31 / 32 events
32 / 32 events


,SOL_standard,event_starttime,event_endtime,event_peaktime,fl_peakflux,fl_peakfluxunit,fl_goescls,hpc_bbox,hpc_coord,hpc_radius,...,meanenergydensityunit,totalenergydensityunit,totalphotoenergy,totalphotoenergyunit,unsignedflux,magfluxunit,highsheararea,highshearareaunit,ar_hpc_bbox,is_ar
0,SOL2014-01-01T00:11:10L107C097,2014-01-01 00:11:10,2014-01-01 00:50:58,2014-01-01T00:39:34,173.5200,erg/cm/cm/s,NaN,"POLYGON((-998.4 -230.4,-921.6 -230.4,-921.6 -7...",POINT(-960 -115.2),966.887294,...,ergs per cubic centimeter,ergs/cm,2.68468e+30,ergs,13772.5,emx,3.54325e+10,km2,"POLYGON((-942.492 -250.443,-872.154 -231.6024,...",1
1,SOL2014-01-01T01:24:46L177C104,2014-01-01 01:24:46,2014-01-01 01:49:58,2014-01-01T01:34:10,105.4170,erg/cm/cm/s,NaN,"POLYGON((-307.2 -307.2,-76.8 -307.2,-76.8 0,-3...",POINT(-192 -192),271.529004,...,ergs per cubic centimeter,ergs/cm,2.04595e+30,ergs,10584.9,emx,3.69705e+10,km2,"POLYGON((-456.27 -332.9994,207.5634 -327.9516,...",1
2,SOL2014-01-01T01:27:24L172C104,2014-01-01 01:27:24,2014-01-01 01:49:12,2014-01-01T01:36:48,859.6840,erg/cm/cm/s,NaN,"POLYGON((-307.2 -307.2,-76.8 -307.2,-76.8 0,-3...",POINT(-268.8 -192),330.329290,...,ergs per cubic centimeter,ergs/cm,2.04595e+30,ergs,10584.9,emx,3.69705e+10,km2,"POLYGON((-456.27 -332.9994,207.5634 -327.9516,...",1
3,SOL2014-01-01T01:29:56L177C099,2014-01-01 01:29:56,2014-01-01 01:39:08,2014-01-01T01:36:32,267.8040,erg/cm/cm/s,NaN,"POLYGON((-230.4 -153.6,-153.6 -153.6,-153.6 -7...",POINT(-192 -115.2),223.908553,...,None,None,None,None,None,None,None,None,None,0
4,SOL2014-01-01T02:26:10L232C103,2014-01-01 02:26:10,2014-01-01 02:31:34,2014-01-01T02:28:58,37.3357,erg/cm/cm/s,NaN,"POLYGON((614.4 -230.4,691.2 -230.4,691.2 -153....",POINT(652.8 -192),680.449734,...,ergs per cubic centimeter,ergs/cm,2.02554e+31,ergs,36527,emx,3.02221e+11,km2,"POLYGON((207.549 -352.431,659.166 -365.9058,71...",1
5,SOL2014-01-01T03:11:46L231C103,2014-01-01 03:11:46,2014-01-01 03:22:10,2014-01-01T03:14:58,44.0679,erg/cm/cm/s,NaN,"POLYGON((614.4 -230.4,691.2 -230.4,691.2 -153....",POINT(652.8 -192),680.449734,...,ergs per cubic centimeter,ergs/cm,2.02554e+31,ergs,36527,emx,3.02221e+11,km2,"POLYGON((207.549 -352.431,659.166 -365.9058,71...",1
6,SOL2014-01-01T03:13:36L225C103,2014-01-01 03:13:36,2014-01-01 03:20:00,2014-01-01T03:14:48,769.3830,erg/cm/cm/s,NaN,"POLYGON((537.6 -230.4,614.4 -230.4,614.4 -153....",POINT(576 -192),607.157311,...,ergs per cubic centimeter,ergs/cm,2.02554e+31,ergs,36527,emx,3.02221e+11,km2,"POLYGON((207.549 -352.431,659.166 -365.9058,71...",1
7,SOL2014-01-01T04:38:48L104C097,2014-01-01 04:38:48,2014-01-01 04:43:24,2014-01-01T04:40:00,818.3590,erg/cm/cm/s,NaN,"POLYGON((-998.4 -153.6,-921.6 -153.6,-921.6 -7...",POINT(-960 -115.2),966.887294,...,ergs per cubic centimeter,ergs/cm,1.01922e+31,ergs,20456.1,emx,9.48195e+10,km2,"POLYGON((-941.082 -255.6648,-857.022 -235.1814...",1
8,SOL2014-01-01T04:39:46L107C092,2014-01-01 04:39:46,2014-01-01 04:44:58,2014-01-01T04:40:46,81.9280,erg/cm/cm/s,NaN,"POLYGON((-998.4 -153.6,-921.6 -153.6,-921.6 0,...",POINT(-960 -38.4),960.767693,...,None,None,None,None,None,None,None,None,None,0
9,SOL2014-01-01T05:45:34L213C104,2014-01-01 05:45:34,2014-01-01 05:52:58,2014-01-01T05:50:34,26.2391,erg/cm/cm/s,NaN,"POLYGON((384 -230.4,460.8 -230.4,460.8 -153.6,...",POINT(422.4 -192),463.988965,...,ergs per cubic centimeter,ergs/cm,1.98381e+31,ergs,38758.5,emx,2.96255e+11,km2,"POLYGON((242.5548 -352.6068,686.046 -367.281,7...",1


In [ ]:
#     ar_list.loc[:, 'start_time'] = datetime.datetime(ar_list['start_year'], ar_list['start_month'], ar_list['start_day'])
#     ar_list.loc[:, 'end_time'] = datetime.datetime(ar_list['end_year'], ar_list['end_month'], ar_list['end_day'])
    ar_list.loc[:, 'start_time'] = map(lambda x, y, z: datetime.datetime(x, y, z), ar_list['start_year'], ar_list['start_month'], 
                                                                        ar_list['start_day'])
    ar_list.loc[:, 'end_time'] = map(lambda x, y, z: datetime.datetime(x, y, z), ar_list['end_year'], ar_list['end_month'], 
                                                                        ar_list['end_day'])